In [ ]:
%load_ext lab_black
import pandas as pd
import numpy as np
import helper

In [ ]:
g = helper.parse_mikenet_sims("all_grain_fillIn2.txt", "grain")
s = helper.parse_mikenet_sims("all_strain_fillIn2.txt", "strain")
new_runs = pd.concat([g, s], ignore_index=True)
new_runs.sample(3)

In [ ]:
# check h-params and n
for x in ["Hidden", "Pnoise", "Epsilon", "PhoHid"]:
    print(f"{x}: {new_runs[x].unique()}")

n = len(new_runs.ID.unique())
print(f"n={n}")

In [ ]:
helper.count_grid(new_runs, ["Hidden", "Pnoise", "Epsilon", "PhoHid"], "raw")

In [ ]:
# Read canonical df 
df_2395 = pd.read_csv("../sims/2395_sims.csv", index_col=0)
df_2395.sample(3)

### Check if duplicate ID exist in new runs

In [ ]:
np.intersect1d(df_2395.ID.unique(), new_runs.ID.unique())
# There are duplicate between 1695 and new_runs, double check content

In [ ]:
n + 2395

In [ ]:
# Make new canonical df
df_2920 = pd.concat([df_2395, new_runs], ignore_index=True)
df_2920.to_csv("../sims/2920_sims.csv")

In [ ]:
helper.count_grid(df_2920, ["Hidden", "PhoHid", "Pnoise", "Epsilon"], "raw")
# Min at needed space n = 7

### Proper grid (n=525)

In [ ]:
df = df_2920.copy()

In [ ]:
# Make a ID only h-param dataset
v = ["ID", "Hidden", "PhoHid", "Pnoise", "Epsilon"]
hpar = df[v].groupby("ID").mean().reset_index()
hpar["Epsilon"] = hpar.Epsilon.round(3)  # Fix Epsilon rounding problem after mean

In [ ]:
# Get 7 ID from each cell in desired grid
n = 10
df = pd.DataFrame()

for hidden in [25, 50, 75, 100, 150, 200, 250]:
    for p_noise in [0, 2, 4, 6, 8]:
        for epsilon in [0.002, 0.004, 0.006, 0.008, 0.010]:
            tmp = hpar.loc[
                (hpar.Hidden == hidden)
                & (hpar.Pnoise == p_noise)
                & (hpar.Epsilon == epsilon)
                & (hpar.PhoHid == 20)
            ]

            df = df.append(tmp.sample(n), ignore_index=True)

In [ ]:
# Check len == n * noise_lvs * hidden_lvs * lr_lvs
assert len(df) == 10 * 7 * 5 * 5
print(len(df))

In [ ]:
sel = df_2920.loc[
    df_2920.ID.isin(df.ID),
]

In [ ]:
sel.to_csv("../sims/part3_1750.csv")

In [ ]:
helper.count_grid(sel, ["Hidden", "PhoHid", "Pnoise", "Epsilon"], "raw")